In [ ]:
from google.colab import drive
drive.mount('/content/drive')

rawdata_path = '/content/drive/MyDrive/my_project/my_project1/rawdata'
data_path = '/content/drive/MyDrive/my_project/my_project1/data'

# rawdata_path = 'rawdata/'
# data_path = 'data/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import re
import os

import geopandas as gpd



In [ ]:
# 인허가 정보 파일 확인
file_list = os.listdir(f'{rawdata_path}/서울시 1인가구/')
file_list

['2022.1월_10개 관심집단수.xlsx',
 '2022.2월_10개 관심집단수.xlsx',
 '2022.3월_10개 관심집단수.xlsx',
 '2022.4월_10개 관심집단수.xlsx',
 '2022.6월_10개 관심집단수.xlsx',
 '2022.5월_10개 관심집단수.xlsx',
 '2022.7월_10개 관심집단수.xlsx',
 '2022.9월_10개 관심집단수.xlsx',
 '2022.8월_10개 관심집단수.xlsx',
 '2022.10월_10개 관심집단 수.xlsx',
 '2022.11월_10개 관심집단 수.xlsx',
 '2022.12월_10개 관심집단 수.xlsx',
 '2023.1월_10개 관심집단 수.xlsx',
 '2023.2월_10개 관심집단 수.xlsx',
 '2023.3월_10개 관심집단 수.xlsx',
 '2023.4월_10개 관심집단 수.xlsx',
 '2023.5월_10개 관심집단 수.xlsx',
 '2023.6월_10개 관심집단 수.xlsx',
 '2023.7월_10개 관심집단 수.xlsx',
 '2023.8월_10개 관심집단 수.xlsx',
 '2023.9월_10개 관심집단 수.xlsx',
 '2023.10월_10개 관심집단 수.xlsx',
 '2023.12월_10개 관심집단 수.xlsx',
 '2023.11월_10개 관심집단 수.xlsx',
 '2024.1월_10개 관심집단 수.xlsx',
 '2

In [ ]:
file_list = file_list[:-1]
file_list

['2022.1월_10개 관심집단수.xlsx',
 '2022.2월_10개 관심집단수.xlsx',
 '2022.3월_10개 관심집단수.xlsx',
 '2022.4월_10개 관심집단수.xlsx',
 '2022.6월_10개 관심집단수.xlsx',
 '2022.5월_10개 관심집단수.xlsx',
 '2022.7월_10개 관심집단수.xlsx',
 '2022.9월_10개 관심집단수.xlsx',
 '2022.8월_10개 관심집단수.xlsx',
 '2022.10월_10개 관심집단 수.xlsx',
 '2022.11월_10개 관심집단 수.xlsx',
 '2022.12월_10개 관심집단 수.xlsx',
 '2023.1월_10개 관심집단 수.xlsx',
 '2023.2월_10개 관심집단 수.xlsx',
 '2023.3월_10개 관심집단 수.xlsx',
 '2023.4월_10개 관심집단 수.xlsx',
 '2023.5월_10개 관심집단 수.xlsx',
 '2023.6월_10개 관심집단 수.xlsx',
 '2023.7월_10개 관심집단 수.xlsx',
 '2023.8월_10개 관심집단 수.xlsx',
 '2023.9월_10개 관심집단 수.xlsx',
 '2023.10월_10개 관심집단 수.xlsx',
 '2023.12월_10개 관심집단 수.xlsx',
 '2023.11월_10개 관심집단 수.xlsx',
 '2024.1월_10개 관심집단 수.xlsx',
 '2

In [ ]:
group1 = ['total_all', 'solo_all']
group2 = [x for x in range(20, 80, 5)]

age_groups = group1 + group2

solo = pd.DataFrame()

for file_name in file_list:

    file_date = re.sub(r'\D', '', file_name)[:-2]

    folder_path = f'{rawdata_path}/서울시 1인가구/'
    path = folder_path + file_name

    temp = pd.read_excel(path)

    temp = temp.iloc[:, :7]

    ## 1인 가구수, 총인구수 소수점 단위 반올림
    temp['1인가구수'] = temp['1인가구수'].apply(lambda x: round(x))
    temp['총인구'] = temp['총인구'].apply(lambda x: round(x))


    temp_all = temp.groupby(['행정동코드'])[['총인구', '1인가구수']].agg('sum')
    temp_all.columns = [f'total_all_{file_date}', f'solo_all_{file_date}']

    age_groups = [x for x in range(20, 80, 5)]

    for age in age_groups:
        temp_age = temp[temp['연령대'].isin([age])]
        temp_group = temp_age.groupby(['행정동코드'])[['총인구', '1인가구수']].agg('sum')
        temp_group.columns = [f'total_{age}_{file_date}', f'solo_{age}_{file_date}']
        temp_all = pd.concat([temp_all, temp_group], axis = 1)

    temp_all.reset_index(inplace = True, drop = False)



    globals()[f'solo{file_date}'] = temp_all.merge(temp[['행정동코드', '자치구', '행정동명']],
                    how = 'left',
                    on = '행정동코드')

    globals()[f'solo{file_date}']['행정동명'] = globals()[f'solo{file_date}']['행정동명'].apply(lambda x: re.sub(r'\·', '', x))

    globals()[f'solo{file_date}'].drop_duplicates(inplace = True)
    globals()[f'solo{file_date}'].reset_index(drop = True, inplace = True)
    globals()[f'solo{file_date}'].to_csv(f'{data_path}/solo/solo{file_date}.csv', index = False)

    solo = pd.concat([solo, globals()[f'solo{file_date}']], axis = 1)

In [ ]:
solo

,행정동코드,total_all_20221,solo_all_20221,total_20_20221,solo_20_20221,total_25_20221,solo_25_20221,total_30_20221,solo_30_20221,total_35_20221,...,total_60_20242,solo_60_20242,total_65_20242,solo_65_20242,total_70_20242,solo_70_20242,total_75_20242,solo_75_20242,자치구,행정동명
0,1101053,7848,1854,582,183,687,220,779,244,758,...,697,116,617,119,445,88,871,287,종로구,사직동
1,1101054,2384,831,156,78,172,90,193,90,211,...,220,54,178,44,143,36,257,101,종로구,삼청동
2,1101055,9116,1530,1147,201,841,154,711,181,652,...,685,72,536,127,394,94,759,225,종로구,부암동
3,1101056,15369,1604,1269,132,1173,127,1172,144,1075,...,1267,98,1003,96,879,87,1452,424,종로구,평창동
4,1101057,6356,798,460,50,426,50,455,67,555,...,479,50,398,84,285,58,538,179,종로구,무악동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,1125070,14560,5147,1657,889,1326,721,1160,218,902,...,841,451,728,95,419,50,361,103,강동구,둔촌1동
420,1125071,22511,2766,1918,248,1999,293,2216,244,2132,...,2045,224,1508,252,1040,174,1496,566,강동구,둔촌2동
421,1125072,29872,4808,2354,271,2464,312,3203,592,3149,...,2568,425,1997,404,1397,287,1937,743,강동구,암사1동
422,1125073,30513,7045,2082,447,2718,641,3443,1039,3027,...,2717,522,2188,516,1516,363,2308,829,강동구,천호2동


In [ ]:
solo[solo.isna().any(axis=1)]

,행정동코드,total_all_20221,solo_all_20221,total_20_20221,solo_20_20221,total_25_20221,solo_25_20221,total_30_20221,solo_30_20221,total_35_20221,...,total_60_20242,solo_60_20242,total_65_20242,solo_65_20242,total_70_20242,solo_70_20242,total_75_20242,solo_75_20242,자치구,행정동명


In [ ]:
solo.to_csv(f'{data_path}/solo/solo.csv')

In [ ]:
selected_csv = ['20221', '20231', '20241']

solo_selected = pd.DataFrame()

for idx, csv in enumerate(selected_csv):
    if idx == 0:
        solo_selected = globals()[f'solo{csv}']
    else :
        solo_selected  = solo_selected.merge(globals()[f'solo{csv}'].drop(columns = ['자치구', '행정동명']),
                                             how = 'left',
                                             on = '행정동코드')


In [ ]:
solo_selected = solo_selected.rename(columns=lambda x: x.replace('20221', '2022'))
solo_selected = solo_selected.rename(columns=lambda x: x.replace('20231', '2023'))
solo_selected = solo_selected.rename(columns=lambda x: x.replace('20241', '2024'))

In [ ]:
solo_selected.to_csv(f'{data_path}/solo/solo_selected.csv')

In [ ]:
solo_selected

,행정동코드,total_all_2022,solo_all_2022,total_20_2022,solo_20_2022,total_25_2022,solo_25_2022,total_30_2022,solo_30_2022,total_35_2022,...,total_55_2024,solo_55_2024,total_60_2024,solo_60_2024,total_65_2024,solo_65_2024,total_70_2024,solo_70_2024,total_75_2024,solo_75_2024
0,1101053,7848,1854,582,183,687,220,779,244,758,...,855,135,714,118,631,121,462,89,847,276
1,1101054,2384,831,156,78,172,90,193,90,211,...,256,71,216,51,176,43,136,34,260,101
2,1101055,9116,1530,1147,201,841,154,711,181,652,...,839,80,674,70,525,124,401,94,738,218
3,1101056,15369,1604,1269,132,1173,127,1172,144,1075,...,1614,109,1247,95,981,92,868,83,1436,412
4,1101057,6356,798,460,50,426,50,455,67,555,...,625,58,472,49,393,82,291,59,522,170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,1125070,14560,5147,1657,889,1326,721,1160,218,902,...,1312,647,1088,583,718,93,458,54,324,93
420,1125071,22511,2766,1918,248,1999,293,2216,244,2132,...,2515,258,2013,219,1470,247,1013,172,1526,565
421,1125072,29872,4808,2354,271,2464,312,3203,592,3149,...,3041,465,2527,416,1894,380,1316,267,1848,702
422,1125073,30513,7045,2082,447,2718,641,3443,1039,3027,...,3299,607,2675,507,2095,489,1501,355,2215,791


# 지도파일과 결합

In [ ]:
def merge_map(df_name, key):

    region_code = pd.read_csv(f'{data_path}/region_code.csv')
    dong_shp = gpd.read_file(f'{rawdata_path}/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')

    # 코드명 타입 일치
    region_code['ADM_CD'] = region_code['ADM_CD'].astype('int')
    dong_shp['ADM_CD'] = dong_shp['ADM_CD'].astype('int')

    # 지도파일과 지역코드 결합
    map_base = dong_shp.merge(region_code,
                          how = 'left',
                          on = 'ADM_CD')
    map_base['ADM_NM'] = map_base['ADM_NM'].apply(lambda x: re.sub(r'\·', '', x))


    if '구동' in df_name.columns:
        pass
    else :
        df_name['구동'] = df_name.apply(lambda row: row['자치구'] + ' ' + row['행정동명'], axis=1)

    df_name.drop(columns = ['자치구', '행정동명'], inplace = True)

    globals()['final'] = map_base.merge(df_name.rename(columns = {'구동' : key}),
                                        how = 'left',
                                        on = key)

    final.dropna(inplace = True)

In [ ]:
merge_map(solo_selected.drop(columns= '행정동코드'), 'key1')
selected_columns = ['ADM_CD'] + final.columns[10:].tolist()
final = final[selected_columns]
final

,ADM_CD,total_all_2022,solo_all_2022,total_20_2022,solo_20_2022,total_25_2022,solo_25_2022,total_30_2022,solo_30_2022,total_35_2022,...,total_55_2024,solo_55_2024,total_60_2024,solo_60_2024,total_65_2024,solo_65_2024,total_70_2024,solo_70_2024,total_75_2024,solo_75_2024
0,11010530,7848.0,1854.0,582.0,183.0,687.0,220.0,779.0,244.0,758.0,...,855.0,135.0,714.0,118.0,631.0,121.0,462.0,89.0,847.0,276.0
1,11010540,2384.0,831.0,156.0,78.0,172.0,90.0,193.0,90.0,211.0,...,256.0,71.0,216.0,51.0,176.0,43.0,136.0,34.0,260.0,101.0
2,11010550,9116.0,1530.0,1147.0,201.0,841.0,154.0,711.0,181.0,652.0,...,839.0,80.0,674.0,70.0,525.0,124.0,401.0,94.0,738.0,218.0
3,11010560,15369.0,1604.0,1269.0,132.0,1173.0,127.0,1172.0,144.0,1075.0,...,1614.0,109.0,1247.0,95.0,981.0,92.0,868.0,83.0,1436.0,412.0
4,11030740,18531.0,5332.0,1138.0,439.0,1829.0,748.0,2393.0,897.0,2037.0,...,1697.0,304.0,1397.0,258.0,1396.0,280.0,895.0,179.0,1441.0,524.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,11010600,3981.0,1120.0,254.0,87.0,313.0,113.0,384.0,146.0,364.0,...,342.0,63.0,347.0,67.0,288.0,84.0,161.0,46.0,395.0,147.0
422,11010610,6809.0,3345.0,556.0,372.0,651.0,431.0,589.0,320.0,496.0,...,894.0,360.0,805.0,331.0,663.0,274.0,403.0,168.0,608.0,299.0
423,11220630,22224.0,3095.0,1948.0,319.0,2129.0,379.0,2448.0,448.0,2070.0,...,2281.0,187.0,1940.0,169.0,1542.0,252.0,827.0,136.0,1284.0,389.0
424,11090700,19773.0,4826.0,1736.0,592.0,1827.0,640.0,1815.0,524.0,1699.0,...,2099.0,402.0,1613.0,311.0,1330.0,370.0,963.0,276.0,1548.0,524.0


In [ ]:
final.to_csv(f'{data_path}/final/singlehousehold.csv', index = False)